In [12]:
# load in csvOut.csv data (holds image name & classification)
import csv
import os #to get all picture in directory
from PIL import Image #to open images
import numpy as np #load in numpy module
import sys

# loads in full csvOut data (name & classification)
with open('..//..//annotations_handheld_exclude_test.csv', newline='') as csvfile:
    cls_full = list(csv.reader(csvfile))

In [13]:
# Get the list of all files in directory
path = "..//..//images_handheld_resized"
dir_list = os.listdir(path)

# Initialize dictionaries to hold bitmap information for each image
rgb_avg = {}
rgb_list = []
cls = []

# loop through files and get bit map for each (save as object where filename => bitmap for r,g,b)
for file in cls_full:
    
    # push classification to list
    cls.append(file[1])
    
    # method found https://stackoverflow.com/questions/46385999/transform-an-image-to-a-bitmap
    img = Image.open(path + "\\" + file[0])
    
    # resize image
    smallest_img = img.resize((120, 80))
    ary = np.array(smallest_img)
    
    # Split the three channels
    r, g, b = np.split(ary,3,axis=2)
    
    # flatten each out
    r = r.reshape(-1)
    g = g.reshape(-1)
    b = b.reshape(-1)
    
    # init list
    rgb_avg[file[0]] = []
    
    # loop through bitmap and take average of 3
    for i in range(len(r)):
        avg = int((r[i] + g[i] + b[i]) / 3)
        rgb_avg[file[0]].append(avg)
    
    rgb_list.append(rgb[file[0]])
    
    # convert array to string
    #rgb_string[file] = np.array2string(rgb[file], precision=2, separator=',', suppress_small=True)


C:\Users\alex.borchers\AppData\Local\Temp\ipykernel_58540\2276476085.py:36: RuntimeWarning: overflow encountered in ubyte_scalars
  avg = int((r[i] + g[i] + b[i]) / 3)


In [14]:
rgb_avg['DSC00026.JPG']

[62,
 70,
 75,
 77,
 77,
 80,
 1,
 0,
 77,
 72,
 68,
 69,
 66,
 60,
 60,
 56,
 61,
 44,
 39,
 37,
 36,
 33,
 22,
 42,
 40,
 33,
 34,
 33,
 35,
 33,
 34,
 34,
 38,
 47,
 51,
 53,
 59,
 68,
 73,
 68,
 66,
 62,
 62,
 60,
 59,
 69,
 64,
 44,
 54,
 63,
 64,
 66,
 77,
 27,
 23,
 60,
 67,
 18,
 82,
 71,
 65,
 47,
 45,
 53,
 62,
 55,
 60,
 62,
 55,
 54,
 60,
 65,
 15,
 47,
 18,
 20,
 67,
 41,
 79,
 79,
 10,
 3,
 65,
 61,
 69,
 70,
 65,
 49,
 54,
 1,
 0,
 5,
 72,
 75,
 79,
 79,
 63,
 61,
 47,
 50,
 65,
 78,
 74,
 66,
 72,
 58,
 62,
 77,
 74,
 74,
 64,
 53,
 59,
 65,
 62,
 63,
 52,
 57,
 58,
 47,
 69,
 74,
 79,
 83,
 81,
 79,
 78,
 82,
 2,
 83,
 74,
 67,
 62,
 58,
 57,
 54,
 53,
 56,
 50,
 44,
 40,
 37,
 33,
 39,
 49,
 36,
 35,
 33,
 34,
 34,
 35,
 40,
 45,
 47,
 52,
 55,
 62,
 74,
 78,
 66,
 68,
 65,
 57,
 59,
 57,
 69,
 7,
 73,
 54,
 63,
 84,
 75,
 74,
 2,
 22,
 73,
 81,
 79,
 4,
 68,
 58,
 49,
 50,
 55,
 53,
 58,
 74,
 70,
 67,
 73,
 76,
 72,
 58,
 4,
 7,
 23,
 42,
 35,
 5,
 0,
 1,
 73,
 68,


In [15]:
# convert bitmaps to indexed np array
rgb_arr = np.array([rgb[row] for row in rgb])
rgb_arr2 = np.array(rgb_list)

# convert classifications to numpy array
cls_np = np.array(cls)

In [16]:
# Training model (load necessary libraries)
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV

#split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(rgb_arr2, cls_np, test_size=0.2)

In [17]:
from sklearn.metrics import accuracy_score

svc = svm.SVC(C=0.1, gamma=0.0001, kernel='poly', probability=True)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.5523255813953488

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(svm.SVC(C=0.1, gamma=0.0001, kernel='poly', probability=True), X_train, y_train, cv=3)
accuracies